# Notebook 2.2: The Kalman Filter for LMPC – Estimating States from Noisy Data

In previous notebooks, our LMPC implementations assumed that the full state vector $x_k$ of the system was perfectly known at each time step. In reality, this is rarely the case. We typically only have access to noisy measurements of some output variables $y_k$, and the system itself is subject to unmeasured process disturbances.

The **Kalman Filter (KF)** is an optimal recursive algorithm for estimating the internal states of a linear dynamic system from a series of noisy measurements. It plays a crucial role in enabling practical Model Predictive Control by providing the necessary state estimates.

**Goals of this Notebook:**
1. Understand the discrete-time Kalman Filter equations (predict and update steps).
2. Implement a Kalman Filter in Python.
3. Discuss the tuning of the process noise covariance ($Q_K$) and measurement noise covariance ($R_K$) matrices.
4. Integrate the Kalman Filter with the LMPC controller (from Notebook 1.3/1.4) to achieve output feedback control.
5. Simulate the LMPC with KF on the double integrator system, visualizing true states, estimated states, and closed-loop performance.

## 1. Importing Libraries and Re-using Code

We'll use NumPy, Matplotlib, SciPy for system definitions, and CVXPY for the MPC's QP solver. We'll adapt the LMPC setup from previous notebooks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import cont2discrete
import cvxpy as cp

# Optional: for nicer plots
plt.rcParams.update({'font.size': 12, 'figure.figsize': (12, 9)})

# --- System Definition (Double Integrator from previous notebooks) ---
Ac = np.array([[0, 1], [0, 0]])
Bc = np.array([[0], [1]])
Cc = np.array([[1, 0]]) # Output is position
Dc = np.array([[0]])
Ts = 0.1
Ad, Bd, Cd, Dd, _ = cont2discrete((Ac, Bc, Cc, Dc), Ts, method='zoh')
n_states = Ad.shape[0]
n_inputs = Bd.shape[1]
n_outputs = Cd.shape[0]

# --- Prediction Matrices Function (from Notebook 1.1) ---
def get_prediction_matrices(Ad_sys, Bd_sys, Cd_sys, Np_horizon):
    n_s, n_i, n_o = Ad_sys.shape[0], Bd_sys.shape[1], Cd_sys.shape[0]
    F = np.zeros((n_o * Np_horizon, n_s))
    Phi = np.zeros((n_o * Np_horizon, n_i * Np_horizon))
    for i in range(Np_horizon):
        F[i*n_o:(i+1)*n_o, :] = Cd_sys @ np.linalg.matrix_power(Ad_sys, i + 1)
        for j in range(i + 1):
            if i-j == 0:
                 Phi[i*n_o:(i+1)*n_o, j*n_i:(j+1)*n_i] = Cd_sys @ Bd_sys
            else:
                 Phi[i*n_o:(i+1)*n_o, j*n_i:(j+1)*n_i] = \
                    Cd_sys @ np.linalg.matrix_power(Ad_sys, i-j) @ Bd_sys
    return F, Phi

# --- QP Components Builder (from Notebook 1.4, simplified for clarity here) ---
def build_qp_components_basic(F_mat, Phi_mat, Q_val, R_val, S_val, Np_h, m_ins, p_outs):
    Q_matrix_s = Q_val * np.eye(p_outs); R_matrix_s = R_val * np.eye(m_ins); S_matrix_s = S_val * np.eye(m_ins)
    Q_bar_s = np.kron(np.eye(Np_h), Q_matrix_s); R_bar_s = np.kron(np.eye(Np_h), R_matrix_s); S_bar_s = np.kron(np.eye(Np_h), S_matrix_s)
    T_diff_s = np.eye(Np_h * m_ins) - np.eye(Np_h * m_ins, k=-m_ins) # Efficient way for T_diff
    T_diff_s[0:m_ins, 0:m_ins] = np.eye(m_ins) # First block of DeltaU is U_k - U_prev
    # To be precise about T_diff for U_k - U_k-1, the first row block of T_diff would be [I 0 ... 0]
    # and U_prev_stacked has u_k-1 in first block. This is simpler: 
    # DeltaU_0 = U_0 - u_prev ; DeltaU_i = U_i - U_i-1 for i>0
    # Create T_prime such that T_prime @ [u_prev; U_k] = Delta_U_k
    # Or, use T_diff as defined in 1.2, and subtract U_prev_stacked (vector with u_prev at top)
    # For build_qp_components in 1.4, we had explicit T_diff and U_prev_stacked creation.
    # Let's reuse that structure for consistency.
    T_diff_s_orig = np.zeros((Np_h * m_ins, Np_h * m_ins))
    for i in range(Np_h):
        T_diff_s_orig[i*m_ins:(i+1)*m_ins, i*m_ins:(i+1)*m_ins] = np.eye(m_ins)
        if i > 0:
            T_diff_s_orig[i*m_ins:(i+1)*m_ins, (i-1)*m_ins:i*m_ins] = -np.eye(m_ins)

    H_qp_s = 2 * (Phi_mat.T @ Q_bar_s @ Phi_mat + R_bar_s + T_diff_s_orig.T @ S_bar_s @ T_diff_s_orig)
    
    def compute_f_qp_inner(xk_curr, R_traj_v, u_prev_curr):
        U_prev_stacked_s = np.zeros((Np_h * m_ins, 1))
        U_prev_stacked_s[0:m_ins, 0] = u_prev_curr.flatten()
        error_term_Fx_R_s = F_mat @ xk_curr - R_traj_v
        f_qp_s = 2 * (Phi_mat.T @ Q_bar_s @ error_term_Fx_R_s - T_diff_s_orig.T @ S_bar_s @ U_prev_stacked_s)
        return f_qp_s
    return H_qp_s, compute_f_qp_inner, T_diff_s_orig # Return T_diff for constraints

# MPC Parameters (from Notebook 1.4)
Np = 20; Qw = 100.0; Rw = 0.1; Sw = 1.0
F_mpc, Phi_mpc = get_prediction_matrices(Ad, Bd, Cd, Np)
H_qp_mpc, compute_f_qp_mpc_func, T_diff_mpc = \
    build_qp_components_basic(F_mpc, Phi_mpc, Qw, Rw, Sw, Np, n_inputs, n_outputs)

## 2. The Kalman Filter Algorithm

The system with noise is described by:
$$ x_{k+1} = A x_k + B u_k + G w_k \quad (w_k \sim \mathcal{N}(0, Q_K)) $$
$$ y_{m,k} = C x_k + D u_k + v_k \quad (v_k \sim \mathcal{N}(0, R_K)) $$

The Kalman Filter operates in two steps:

**1. Time Update (Prediction):**
   $\hat{x}_{k|k-1} = A \hat{x}_{k-1|k-1} + B u_{k-1}$ (Predict state)
   $P_{k|k-1} = A P_{k-1|k-1} A^T + G Q_K G^T$ (Predict error covariance)

**2. Measurement Update (Correction):**
   $K_k = P_{k|k-1} C^T (C P_{k|k-1} C^T + R_K)^{-1}$ (Compute Kalman Gain)
   $\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k (y_{m,k} - (C \hat{x}_{k|k-1} + D u_k))$ (Update state estimate)
   $P_{k|k} = (I - K_k C) P_{k|k-1}$ (Update error covariance)

Here, $G$ is the process noise input matrix. If process noise is assumed to affect each state directly, $G$ can be $I$ (identity matrix) or chosen based on knowledge of how disturbances enter the system.

In [ ]:
class KalmanFilter:
    def __init__(self, Ad, Bd, Cd, Dd, G_mat, Qk_mat, Rk_mat, x_hat0, P0):
        self.Ad = Ad
        self.Bd = Bd
        self.Cd = Cd
        self.Dd = Dd
        self.G = G_mat # Process noise input matrix
        self.Qk = Qk_mat # Process noise covariance
        self.Rk = Rk_mat # Measurement noise covariance
        
        self.x_hat = x_hat0 # Current state estimate (a posteriori x_k|k)
        self.P = P0       # Current error covariance (a posteriori P_k|k)
        
        self.n_states = Ad.shape[0]

    def predict(self, uk_minus_1):
        # Time Update (Prediction)
        # uk_minus_1 is the input applied at the previous step that led to current state x_k
        self.x_hat_apriori = self.Ad @ self.x_hat + self.Bd @ uk_minus_1
        self.P_apriori = self.Ad @ self.P @ self.Ad.T + self.G @ self.Qk @ self.G.T
        return self.x_hat_apriori
        
    def update(self, y_measured_k, uk_current):
        # Measurement Update (Correction)
        # y_measured_k is the measurement at time k
        # uk_current is the input applied at time k if D is non-zero
        
        # Compute Kalman Gain
        S_innovation = self.Cd @ self.P_apriori @ self.Cd.T + self.Rk
        Kk = self.P_apriori @ self.Cd.T @ np.linalg.inv(S_innovation)
        
        # Update state estimate
        y_predicted = self.Cd @ self.x_hat_apriori + self.Dd @ uk_current
        innovation = y_measured_k - y_predicted
        self.x_hat = self.x_hat_apriori + Kk @ innovation
        
        # Update error covariance
        I_n = np.eye(self.n_states)
        self.P = (I_n - Kk @ self.Cd) @ self.P_apriori
        # Numerically more stable Joseph form for P update (optional):
        # self.P = (I_n - Kk @ self.Cd) @ self.P_apriori @ (I_n - Kk @ self.Cd).T + Kk @ self.Rk @ Kk.T
        
        return self.x_hat, self.P

    def step(self, uk_prev_for_predict, y_measured_k, uk_curr_for_update):
        """Combines predict and update steps."""
        self.predict(uk_prev_for_predict)
        self.update(y_measured_k, uk_curr_for_update)
        return self.x_hat, self.P

### Tuning $Q_K$ and $R_K$

*   **$R_K$ (Measurement Noise Covariance):** Reflects sensor accuracy. Diagonal elements are variances of individual sensor noises. Can often be estimated from sensor datasheets or by analyzing sensor readings when the true value is known (e.g., variance of readings around a constant value).
*   **$Q_K$ (Process Noise Covariance):** Represents uncertainty in the process model ($A_d, B_d$) and unmeasured disturbances. $G$ maps this abstract noise to how it affects states. $Q_K$ is often a tuning parameter:
    *   **Larger $Q_K$ values:** Imply less trust in the model. The filter will rely more on measurements (Kalman gain $K_k$ will be larger), leading to faster response to real state changes but potentially noisier estimates.
    *   **Smaller $Q_K$ values:** Imply more trust in the model. The filter will smooth out measurements more (Kalman gain $K_k$ will be smaller), leading to less noisy estimates but potentially lagging true state changes if the model is inaccurate or unmeasured disturbances are significant.

Finding the right balance is key. For our double integrator, let's assume $G=I$ initially, meaning process noise affects both position and velocity states directly.

In [ ]:
# Kalman Filter Parameters
G_kf = np.eye(n_states) # Process noise affects states directly
# G_kf = Bd # If process noise enters through input channel

# Measurement noise covariance (for position sensor)
R_k_val = 0.01**2 # Assume std dev of measurement noise is 0.01
Rk_kf = np.array([[R_k_val]]) # p x p matrix (here p=1)

# Process noise covariance (TUNING PARAMETERS)
q_pos_noise_std = 0.005 # std dev of noise on position state 
q_vel_noise_std = 0.05  # std dev of noise on velocity state
Qk_kf = np.diag([q_pos_noise_std**2, q_vel_noise_std**2]) # n x n matrix

# Initial state estimate and covariance for KF
x_hat0_kf = np.array([[0.0], [0.0]]) # Initial guess for state
P0_kf = np.diag([0.1**2, 0.1**2])   # Uncertainty in initial guess

# Instantiate the Kalman Filter
kf = KalmanFilter(Ad, Bd, Cd, Dd, G_kf, Qk_kf, Rk_kf, x_hat0_kf, P0_kf)

## 3. LMPC with Kalman Filter (Output Feedback MPC)

Now we modify the LMPC simulation loop:
1.  At time $k$, the plant is at true state $x_{true,k}$.
2.  A noisy measurement $y_{m,k} = C x_{true,k} + D u_{true,k-1} + v_k$ is obtained.
3.  The Kalman Filter uses $u_{true,k-1}$ (input that led to $x_{true,k}$) for its prediction step, and $y_{m,k}$ and $u_{true,k}$ (if $D \neq 0$, for $y_{m,k}$'s D-term part) for its update step to get $\hat{x}_{k|k}$.
4.  This $\hat{x}_{k|k}$ is passed to the MPC controller as the current state $x_k$.
5.  MPC computes $u_k^* = u_{k|k}^*$.
6.  This $u_k^*$ is applied to the true plant (which also experiences process noise $w_k$) to get $x_{true,k+1}$.
7.  Repeat.

In [ ]:
# --- LMPC with KF Simulation ---
sim_steps_kf = 100

# Plant initial conditions
x_true_plant = np.array([[0.0], [0.0]]) 
u_prev_applied_true = np.array([[0.0]]) # Input applied at k-1 to get x_true_plant at k

# Kalman Filter re-initialization for each simulation run (if tuning)
kf = KalmanFilter(Ad, Bd, Cd, Dd, G_kf, Qk_kf, Rk_kf, x_hat0_kf.copy(), P0_kf.copy())

# MPC reference
setpoint_kf = 1.0
R_traj_target_kf = np.ones((Np * n_outputs, 1)) * setpoint_kf

# Data logging
X_true_log = np.zeros((n_states, sim_steps_kf + 1))
X_hat_log = np.zeros((n_states, sim_steps_kf + 1))
Y_measured_log = np.zeros((n_outputs, sim_steps_kf))
U_applied_log = np.zeros((n_inputs, sim_steps_kf))
P_diag_log = np.zeros((n_states, sim_steps_kf + 1)) # Log diagonal of P_k|k

X_true_log[:, 0] = x_true_plant.flatten()
X_hat_log[:, 0] = kf.x_hat.flatten()
P_diag_log[:, 0] = np.diag(kf.P)

# Get H_qp (constant) and f_qp function for MPC
H_qp_for_kf_sim, compute_f_qp_for_kf_sim, T_diff_for_kf_sim = \
    build_qp_components_basic(F_mpc, Phi_mpc, Qw, Rw, Sw, Np, n_inputs, n_outputs)

# CVXPY QP Solver function (unconstrained for now)
def solve_qp_cvxpy_unconstrained(H_qp_mat, f_qp_vec, solver=cp.OSQP):
    num_vars_U = H_qp_mat.shape[0]
    U_dv = cp.Variable(num_vars_U)
    H_qp_symm = 0.5 * (H_qp_mat + H_qp_mat.T)
    objective = cp.Minimize(0.5 * cp.quad_form(U_dv, H_qp_symm) + f_qp_vec.flatten() @ U_dv)
    problem = cp.Problem(objective)
    problem.solve(solver=solver, verbose=False)
    if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        return U_dv.value.reshape(-1, 1)
    else:
        print(f"QP Solver issue: {problem.status} at a sim step.")
        return np.zeros((num_vars_U,1)) # Fallback: zero control sequence

print(f"Starting LMPC with KF simulation for {sim_steps_kf} steps...")
for k_sim in range(sim_steps_kf):
    print(f"Sim step {k_sim+1}/{sim_steps_kf}", end='\r')
    
    # 1. Plant: Current true state is x_true_plant (which is X_true_log[:, k_sim])
    
    # 2. Measurement: Simulate noisy measurement from true plant state
    # y_m,k = C x_true,k + D u_true,k-1 (if D related to input causing current state)
    # or y_m,k = C x_true,k + D u_true,k (if D related to current input about to be applied - not yet known for KF input)
    # Let's assume D=0, so y_m,k = C x_true,k + v_k
    v_k_noise = np.random.normal(0, np.sqrt(Rk_kf[0,0])) # Measurement noise sample
    y_measured = Cd @ x_true_plant + Dd @ u_prev_applied_true + v_k_noise # y_m,k = C x_true,k + v_k (since Dd=0)
    Y_measured_log[:, k_sim] = y_measured.flatten()
    
    # 3. Kalman Filter: 
    #   uk_prev_for_predict = u_prev_applied_true (input that resulted in x_true_plant)
    #   uk_curr_for_update = u_prev_applied_true (if Dd != 0 in y_m = Cx + Du + v, here Dd=0, so it doesn't matter)
    x_hat_current, P_current = kf.step(u_prev_applied_true, y_measured, u_prev_applied_true) 
    X_hat_log[:, k_sim] = x_hat_current.flatten() # Store x_hat_k|k
    P_diag_log[:, k_sim] = np.diag(P_current)
    
    # 4. MPC: Use estimated state x_hat_current
    f_qp_for_mpc = compute_f_qp_for_kf_sim(x_hat_current, R_traj_target_kf, u_prev_applied_true)
    
    # 5. Solve QP
    U_optimal_mpc = solve_qp_cvxpy_unconstrained(H_qp_for_kf_sim, f_qp_for_mpc)
    
    if U_optimal_mpc is None: # Should have been handled by fallback in solver
        u_applied_true = u_prev_applied_true
    else:
        u_applied_true = U_optimal_mpc[0:n_inputs].reshape(n_inputs, 1)
    
    U_applied_log[:, k_sim] = u_applied_true.flatten()
    
    # 6. Plant Evolves: Apply u_applied_true to true plant with process noise
    w_k_noise = np.random.multivariate_normal([0,0], Qk_kf).reshape(n_states,1) # Process noise sample
    x_true_plant_next = Ad @ x_true_plant + Bd @ u_applied_true + G_kf @ w_k_noise
    
    # Log true state for next iteration
    X_true_log[:, k_sim + 1] = x_true_plant_next.flatten()
    
    # Update for next loop iteration
    x_true_plant = x_true_plant_next
    u_prev_applied_true = u_applied_true

# Final estimate at end of sim
X_hat_log[:, sim_steps_kf] = kf.x_hat.flatten()
P_diag_log[:, sim_steps_kf] = np.diag(kf.P)

print("\nLMPC with KF simulation finished.")

## 4. Visualizing Performance with Kalman Filter

In [ ]:
time_sim_kf = np.arange(0, sim_steps_kf * Ts, Ts)
time_states_kf = np.arange(0, (sim_steps_kf + 1) * Ts, Ts)

plt.figure(figsize=(14, 12))

# Position (State x1 and Output y1)
plt.subplot(3,2,1)
plt.plot(time_states_kf, X_true_log[0,:], 'b-', label='True Position (x1_true)')
plt.plot(time_states_kf, X_hat_log[0,:], 'r--', label='Estimated Position (x1_hat)')
plt.plot(time_sim_kf, Y_measured_log[0,:], 'gx', markersize=4, alpha=0.6, label='Measured Position (y_m)')
plt.axhline(setpoint_kf, color='k', linestyle=':', label='Setpoint')
plt.title(f'Output Feedback LMPC w/ KF (Np={Np}, Qw={Qw}, Rw={Rw}, Sw={Sw})')
plt.ylabel('Position')
plt.grid(True); plt.legend()

# Velocity (State x2)
plt.subplot(3,2,3)
plt.plot(time_states_kf, X_true_log[1,:], 'b-', label='True Velocity (x2_true)')
plt.plot(time_states_kf, X_hat_log[1,:], 'r--', label='Estimated Velocity (x2_hat)')
plt.ylabel('Velocity'); plt.grid(True); plt.legend()

# Control Input
plt.subplot(3,2,5)
plt.step(time_sim_kf, U_applied_log[0,:], 'k-', where='post', label='Control Input u')
plt.ylabel('Input (Force)'); plt.xlabel('Time (s)'); plt.grid(True); plt.legend()

# Estimation Errors
plt.subplot(3,2,2)
plt.plot(time_states_kf, X_true_log[0,:] - X_hat_log[0,:], 'm-', label='Error x1 (Pos)')
plt.plot(time_states_kf, 2*np.sqrt(P_diag_log[0,:]), 'm:', label='+-2*std_dev(x1_hat)')
plt.plot(time_states_kf, -2*np.sqrt(P_diag_log[0,:]), 'm:')
plt.title('State Estimation Errors & Covariance')
plt.ylabel('Position Error'); plt.grid(True); plt.legend()

plt.subplot(3,2,4)
plt.plot(time_states_kf, X_true_log[1,:] - X_hat_log[1,:], 'c-', label='Error x2 (Vel)')
plt.plot(time_states_kf, 2*np.sqrt(P_diag_log[1,:]), 'c:', label='+-2*std_dev(x2_hat)')
plt.plot(time_states_kf, -2*np.sqrt(P_diag_log[1,:]), 'c:')
plt.ylabel('Velocity Error'); plt.xlabel('Time (s)'); plt.grid(True); plt.legend()

plt.tight_layout()
plt.show()

## 5. Experimenting with KF Tuning ($Q_K, R_K$)

The performance of the Kalman Filter, and consequently the output feedback MPC, is sensitive to the choice of $Q_K$ and $R_K$.

*   **If $R_K$ is too small (filter trusts noisy measurements too much):** Estimated states might be noisy, leading to jerky control action.
*   **If $R_K$ is too large (filter trusts measurements too little):** Estimation might be slow to respond to true state changes if the model is not perfect, potentially relying too much on an inaccurate model.
*   **If $Q_K$ is too small (filter trusts model too much):** If there are unmodeled disturbances or model errors, the filter might diverge or be slow to track true state changes.
*   **If $Q_K$ is too large (filter trusts model too little):** Estimates will respond quickly to measurements but might become noisy, similar to having a small $R_K$.

**Exercise:**
1.  Re-run the simulation with the default $Q_K, R_K$.
2.  Try increasing the `R_k_val` significantly (e.g., `0.1**2`). How do the estimates and control action change?
3.  Reset `R_k_val`. Try decreasing `q_pos_noise_std` and `q_vel_noise_std` (elements of $Q_K$) significantly (e.g., by a factor of 10). What happens?
4.  Try increasing `q_pos_noise_std` and `q_vel_noise_std`. Observe the effect on estimate smoothness and tracking of true states.

The plots of estimation error along with the $\pm 2\sigma$ bounds (from the diagonal of $P_{k|k}$) can help assess if the filter is well-tuned. Ideally, the true error should mostly lie within these bounds, and the bounds should not be excessively large or small.

## 6. Key Takeaways

*   The Kalman Filter provides a systematic way to estimate system states from noisy measurements in the presence of process noise.
*   It operates via a recursive predict-correct cycle.
*   Tuning the noise covariance matrices $Q_K$ (process noise) and $R_K$ (measurement noise) is crucial for good estimation performance.
*   Integrating a Kalman Filter with LMPC allows for **output feedback MPC**, where the controller uses estimated states instead of assuming full state knowledge.
*   The quality of state estimation directly impacts the performance of the MPC controller.

This concludes our foundational exploration of LMPC modeling, prediction, optimization, constraint handling, and state estimation. In the next part of the series (**Part 3: Nonlinear MPC**), we will move beyond linear systems and tackle the challenges and opportunities presented by nonlinear dynamics.